In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as sk
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import accuracy_score
%matplotlib inline

In [2]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [3]:
#먼저 데이터 읽어오기
df_KP2020=pd.read_csv('../../../police_data/KP2020.csv',encoding='cp949')
df_KP2021=pd.read_csv('../../../police_data/KP2021.csv',encoding='cp949')
df_NPA2020=pd.read_csv('../../../police_data/NPA2020.csv',encoding='cp949')

df_KP2020.head()

,RECV_DEPT_NM,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
0,충남청,20/12/01 01:43:07.000000000,19,10,305,1.0,충청남도 보령시 궁촌동(행정:대천4동) 369,126.598345,36.341537,Y
1,대전청,20/12/01 02:05:04.000000000,13,10,601,3.0,NaN,127.404663,36.341685,NaN
2,대전청,20/12/01 02:06:52.000000000,13,10,601,1.0,NaN,127.404663,36.341685,NaN
3,충남청,20/12/01 02:37:25.000000000,19,10,606,3.0,충청남도 보령시 천북면 하만리 628-10,126.524980,36.474390,N
4,충남청,20/12/01 08:17:50.000000000,19,10,401,2.0,충청남도 천안시 서북구 성정동(행정:성정2동) 1259,127.137160,36.826718,NaN


In [4]:
df_KP=pd.concat([df_KP2020,df_KP2021], ignore_index=True)

In [5]:
new_df_KP=df_KP.dropna(axis=0)
new_df_KP.isnull().sum()

RECV_DEPT_NM     0
RECV_CPLT_DM     0
NPA_CL           0
EVT_STAT_CD      0
EVT_CL_CD        0
RPTER_SEX        0
HPPN_PNU_ADDR    0
HPPN_X           0
HPPN_Y           0
SME_EVT_YN       0
dtype: int64

In [6]:
new_df_KP["도"]=new_df_KP['HPPN_PNU_ADDR'].str.split(" ", expand=True)[0]
new_df_KP["시"]=new_df_KP['HPPN_PNU_ADDR'].str.split(" ", expand=True)[1]

C:\Users\jaemin\AppData\Local\Temp\ipykernel_5160\3110203428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_KP["도"]=new_df_KP['HPPN_PNU_ADDR'].str.split(" ", expand=True)[0]
C:\Users\jaemin\AppData\Local\Temp\ipykernel_5160\3110203428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_KP["시"]=new_df_KP['HPPN_PNU_ADDR'].str.split(" ", expand=True)[1]


In [7]:
new_df_KP.head()

,RECV_DEPT_NM,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN,도,시
0,충남청,20/12/01 01:43:07.000000000,19,10,305,1.0,충청남도 보령시 궁촌동(행정:대천4동) 369,126.598345,36.341537,Y,충청남도,보령시
3,충남청,20/12/01 02:37:25.000000000,19,10,606,3.0,충청남도 보령시 천북면 하만리 628-10,126.524980,36.474390,N,충청남도,보령시
5,충남청,20/12/01 02:50:39.000000000,19,10,301,2.0,충청남도 아산시 둔포면 석곡리(행정:둔포면) 1707,127.057462,36.920405,Y,충청남도,아산시
6,대전청,20/12/01 03:22:39.000000000,13,10,107,2.0,대전광역시 대덕구 중리동(행정:중리동) 381-25,127.422406,36.363326,Y,대전광역시,대덕구
13,충남청,20/12/01 02:13:23.000000000,19,10,307,1.0,충청남도 아산시 용화동(행정:온양5동) 1089,127.008144,36.776451,Y,충청남도,아산시


In [8]:
new_df_KP=new_df_KP[['RECV_DEPT_NM','도','시','RECV_CPLT_DM','NPA_CL','EVT_STAT_CD','EVT_CL_CD','RPTER_SEX','HPPN_X','HPPN_Y','SME_EVT_YN','HPPN_PNU_ADDR']]

In [9]:
new_df_KP.drop(['HPPN_PNU_ADDR'],axis=1,inplace=True)

In [10]:
new_df_KP.reset_index(inplace=True, drop=True)

In [19]:
new_df_KP.head()

,RECV_DEPT_NM,도,시,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_X,HPPN_Y,SME_EVT_YN
0,충남청,충청남도,보령시,20/12/01 01:43:07.000000000,19,10,305,1.0,126.598345,36.341537,Y
1,충남청,충청남도,보령시,20/12/01 02:37:25.000000000,19,10,606,3.0,126.524980,36.474390,N
2,충남청,충청남도,아산시,20/12/01 02:50:39.000000000,19,10,301,2.0,127.057462,36.920405,Y
3,대전청,대전광역시,대덕구,20/12/01 03:22:39.000000000,13,10,107,2.0,127.422406,36.363326,Y
4,충남청,충청남도,아산시,20/12/01 02:13:23.000000000,19,10,307,1.0,127.008144,36.776451,Y


In [28]:
new_df_KP.to_excel('./result/old_split_city.xlsx')

In [18]:
type(new_df_KP['RECV_CPLT_DM'][0])

str